In [1]:
import urllib.request
import urllib.parse
import re
import spotipy
import spotipy.util as util
import spotify_tokens
import pandas as pd
pd.options.display.max_rows = 2000
user = 'matteo7moh'

In [2]:
tracks = pd.read_csv('titles.csv')

In [2]:
token = util.prompt_for_user_token(user, "user-library-read", 
                            client_id=spotify_tokens.SPOTIPY_CLIENT_ID,
                            client_secret=spotify_tokens.SPOTIPY_CLIENT_SECRET, 
                            redirect_uri=spotify_tokens.SPOTIPY_REDIRECT_URI)

In [3]:
sp = spotipy.Spotify(auth=token)

In [39]:
playlists = sp.user_playlists(user)

In [40]:
for playlist in playlists['items']:
    print(playlist['name'])
#     results = sp.user_playlist(user, playlist['id'],fields="tracks,next")

_schubert
BMW Classics - 1 Jul 2018 - Trafalgar Square, London
Sommernachtskonzert 2018, Schönbrunn Palace Wien
Giants
Sam Rae – Stories from the Marrow
Voices and Rythms
Airplanes
Trains
Son Lux
Dark Journeys
Musikverein Konzert - Beethoven und Prokofiev


In [8]:
results1 = sp.current_user_saved_tracks(limit=5)
results2 = sp.current_user_saved_tracks(limit=5, offset=5)

In [25]:
''.join([c for c in track['name'] if c != ','])

'Peae Piee'

In [26]:
lim = 10
n = 0
for i in range(30):
    results = sp.current_user_saved_tracks(limit=lim, offset=i*lim)
    for item in results['items']:
        track = item['track']
        print('_'.join(track['artists'][0]['name'].split(' ')) + ',' + ''.join([c for c in track['name'] if c != ',']))
        n += 1

Colin_Stetson,To See More Light
Colin_Stetson,Rubdown
Colin_Stetson,Sunrise
Colin_Stetson,At God’s Doorstep
Colin_Stetson,Like wolves on the fold
Colin_Stetson,All this I do for glory
Colin_Stetson,Control Room
Kangding_Ray,Sub.Res
Kangding_Ray,Dadaist
Macklemore,Ain't Gonna Die Tonight (feat. Eric Nally)
Macklemore,Glorious (feat. Skylar Grey)
Macklemore,Marmalade (feat. Lil Yachty)
Macklemore,Willy Wonka (feat. Offset)
Macklemore,Intentions (feat. Dan Caplen)
Macklemore,Good Old Days (feat. Kesha)
Macklemore,Levitate (feat. Otieno Terry)
Macklemore,Firebreather (feat. Reignwolf)
Macklemore,How to Play the Flute (feat. King Draino)
Macklemore,Ten Million
Macklemore,Over It (feat. Donna Missal)
Macklemore,Zara (feat. Abir)
Macklemore,Corner Store (feat. Dave B & Travis Thompson)
Macklemore,Miracle (feat. Dan Caplen)
Macklemore,Church (feat. Xperience)
Macklemore,Excavate (feat. Saint Claire)
Kangding_Ray,Son
Kangding_Ray,Transitional Ballistics
Kangding_Ray,Crystal
Kangding_Ray,History

In [11]:
for i,item in enumerate(results2['items']):
    track = item['track']
    print(i, track['name'] + ' - ' + track['artists'][0]['name'])

0 All this I do for glory - Colin Stetson
1 Control Room - Colin Stetson
2 Sub.Res - Kangding Ray
3 Dadaist - Kangding Ray
4 Ain't Gonna Die Tonight (feat. Eric Nally) - Macklemore


In [9]:
for i,item in enumerate(results['items']):
    track = item['track']
    print(i, track['name'] + ' - ' + track['artists'][0]['name'])

0 To See More Light - Colin Stetson
1 Rubdown - Colin Stetson
2 Sunrise - Colin Stetson
3 At God’s Doorstep - Colin Stetson
4 Like wolves on the fold - Colin Stetson
5 All this I do for glory - Colin Stetson
6 Control Room - Colin Stetson
7 Sub.Res - Kangding Ray
8 Dadaist - Kangding Ray
9 Ain't Gonna Die Tonight (feat. Eric Nally) - Macklemore
10 Glorious (feat. Skylar Grey) - Macklemore
11 Marmalade (feat. Lil Yachty) - Macklemore
12 Willy Wonka (feat. Offset) - Macklemore
13 Intentions (feat. Dan Caplen) - Macklemore
14 Good Old Days (feat. Kesha) - Macklemore
15 Levitate (feat. Otieno Terry) - Macklemore
16 Firebreather (feat. Reignwolf) - Macklemore
17 How to Play the Flute (feat. King Draino) - Macklemore
18 Ten Million - Macklemore
19 Over It (feat. Donna Missal) - Macklemore
20 Zara (feat. Abir) - Macklemore
21 Corner Store (feat. Dave B & Travis Thompson) - Macklemore
22 Miracle (feat. Dan Caplen) - Macklemore
23 Church (feat. Xperience) - Macklemore
24 Excavate (feat. Saint C

In [30]:
def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print(track['artists'][0]['name'],track['name'])

In [31]:
tracks = results['tracks']
show_tracks(tracks)
while tracks['next']:
    tracks = sp.next(tracks)
    show_tracks(tracks)

Franz Schubert Impromptus, D. 899 (Op. 90): Impromptu No. 3 in G-Flat Major: Andante
Franz Schubert Symphony No.5 In B Flat, D.485: 1. Allegro
Franz Schubert Piano Quintet in A Major, Op. 114, D. 667, "Die Forelle" (The Trout): II. Andante
Franz Schubert Schubert: String Quintet in C Major, D. 956: II. Adagio
Franz Schubert Piano sonata in A Major, D. 959: II. Andantino
Franz Schubert Schubert: String Quintet in C Major, D. 956: II. Adagio
Franz Schubert String Quartet in D Minor, Op. posth. D.810 "Death and the Maiden": II. Andante con moto
Franz Schubert Piano Trio No. 2 in E-Flat Major, Op. 100, D. 929: II. Andante con moto
Franz Schubert Piano Trio No. 2 in E-Flat Major, Op. 100, D. 929: II. Andante con moto
Franz Schubert Piano Sonata in B Flat Major, D. 960: I. Molto moderato


In [19]:
[el['tracks'] for el in playlists['items']]

[{'href': 'https://api.spotify.com/v1/playlists/73NHpzwP6FOPjK5ScFiWNd/tracks',
  'total': 10},
 {'href': 'https://api.spotify.com/v1/playlists/5aQjFiT7syPvxRwcJAy5cv/tracks',
  'total': 43},
 {'href': 'https://api.spotify.com/v1/playlists/3Q4kzEeQoDMG4Qwcwf100Z/tracks',
  'total': 13},
 {'href': 'https://api.spotify.com/v1/playlists/0U2uwwq7ER3VSJPwJiIx95/tracks',
  'total': 430},
 {'href': 'https://api.spotify.com/v1/playlists/7AudCeOfLxv37mXmPuiYPA/tracks',
  'total': 10},
 {'href': 'https://api.spotify.com/v1/playlists/6roXPeOTu6rZgq5IcZEEmj/tracks',
  'total': 31},
 {'href': 'https://api.spotify.com/v1/playlists/0mWNRtXZ7WBdrsdWIKwSel/tracks',
  'total': 90},
 {'href': 'https://api.spotify.com/v1/playlists/0uioYoADYkpFXFrHykyQme/tracks',
  'total': 23},
 {'href': 'https://api.spotify.com/v1/playlists/2FBSqjI4UQknipzQwZojJR/tracks',
  'total': 15},
 {'href': 'https://api.spotify.com/v1/playlists/3ZgU192NAvbl3vUboD9i62/tracks',
  'total': 69},
 {'href': 'https://api.spotify.com/v1/p

In [3]:
# cleaning artists
tracks['artists'] = tracks['artists'].apply\
    (lambda x: x[len('\ufeff'):] if x[:len('\ufeff')] == '\ufeff' else x)

In [4]:
# how many distinct artists do I have?
artists = set()
for a in list(tracks['artists'].unique()):
#     print('-------', a)
    names = a.split(' ')
    for n in names:
        artists.add(n)
len(artists)

829

In [5]:
# to check possible typos
sorted_artists = list(artists)
sorted_artists.sort()
# sorted_artists

In [6]:
# what are the most frequent artists?
artist_occurrences = dict()
for a in list(tracks['artists']):
    names = a.split(' ')
    for n in names:
        if n not in artist_occurrences.keys():
            artist_occurrences[n] = 0
        artist_occurrences[n] += 1

In [7]:
artist_occurrences_list = []
for a, o in artist_occurrences.items():
#     print('-',o)
    artist_occurrences_list.append([a, o])
artist_occurrences_list.sort(key=lambda x: -x[1])

In [8]:
artist_occurrences_list[:20]

[['Paul_Kalkbrenner', 44],
 ['Ludovico_Einaudi', 43],
 ['Yann_Tiersen', 40],
 ['Coldplay', 37],
 ['Flume', 29],
 ['Enrico_Caruso', 27],
 ['Chopin', 24],
 ['Fritz_Kalkbrenner', 24],
 ['Nils_Frahm', 21],
 ['Florence_and_The_Machine', 20],
 ['Daughter', 19],
 ['Fabri_Fibra', 19],
 ['Beriot', 18],
 ['Hardwell', 18],
 ['David_Guetta', 17],
 ['Johann_Strauss_II', 16],
 ['Benny_Benassi', 16],
 ['Arvo_Pärt', 16],
 ['Locatelli', 15],
 ['The_xx', 15]]

# listen to tracks

In [9]:
tracks_list = tracks.values.tolist()

In [10]:
def printt(s):
    print('  ' + str(s))


In [11]:
import numpy as np
import math

In [100]:
i -= 1
# i = 1361
# i = len(tracks_list) - 1
t = tracks_list[i]
search = ''
for el in t:
    try:
        if math.isnan(el):
            pass
#             printt(el)
    except:
        el = el.replace('_', ' ')
        search += (el) + ' '
search = search[:-1]
query_string = urllib.parse.urlencode({"search_query" : search})
html_content = urllib.request.urlopen("http://www.youtube.com/results?" + query_string)
search_results = re.findall(r'href=\"\/watch\?v=(.{11})', html_content.read().decode())
print(search)
print("http://www.youtube.com/watch?v=" + search_results[0])

# to get genre and bpm
sb = search + ' ' 'beatport'
sbl = sb.split(' ')
s2 = '+'.join(sbl)
print("https://www.google.com/search?q=" + s2)


Muse Madness
http://www.youtube.com/watch?v=Ek0SgwWmF9w
https://www.google.com/search?q=Muse+Madness+beatport


In [79]:
[s + '+' for s in search.split(' ')] + ['beatport']

['Steve+', 'Parker+', 'The+', 'Prophet+', 'beatport']

artist,title,genre,bpm,src,tags

tags: soft, chill, heavy, piano, voice, synth, brass, strings, orchestra